In [1]:
EMR=False # Set to True if running notebook on EMR

## Computing PCA using RDDs

##  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We will perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### The effect of  `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead of Dataframes.
* Why? Because numpy treats `nan` entries correctly:
  * In numpy `5+nan=5` while in dataframes `5+nan=nan`

### Performing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

In [2]:
if not EMR:
    import findspark
    findspark.init()
from pyspark import SparkContext,SparkConf

def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '2g')
    sc_conf.set('spark.executor.cores', '4')
    sc_conf.set('spark.cores.max', '40')
    sc_conf.set('spark.default.parallelism','10')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

#sc.stop()
#sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])
sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

dict_items([('spark.app.name', 'Weather_PCA'), ('spark.executor.memory', '2g'), ('spark.executor.cores', '4'), ('spark.cores.max', '40'), ('spark.default.parallelism', '10'), ('spark.logConf', 'True')])


In [3]:
from pyspark.sql import *
sqlContext = SQLContext(sc)

import numpy as np
from lib.computeStatistics import *

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into the states of the continental USA (plus a few stations from states in canada and mexico).

In [37]:
!ls /Users/yoavfreund/projects/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data/Weather/

NY.parquet        NY.tgz            Old               STAT_NY.pickle.gz


In [5]:
state='NY'
if not EMR:
    data_dir='../../Data/Weather'
    tarname=state+'.tgz'
    parquet=state+'.parquet'

    !rm -rf $data_dir/$tarname

    command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s"%(tarname,data_dir,tarname)
    print(command)
    !$command
    !ls -lh $data_dir/$tarname

    cur_dir,=!pwd
    %cd $data_dir
    !tar -xzf $tarname
    !du ./$parquet
    %cd $cur_dir

In [8]:
if EMR:  # not debugged, should use complete parquet and extract just the state of interest.
    data_dir='/mnt/workspace/Data'
    !hdfs dfs -mkdir /weather/
    !hdfs dfs -CopyFromLocal $data_dir/$parquet /weather/$parquet

    # When running on cluster
    #!mv ../../Data/Weather/NY.parquet /mnt/workspace/Data/NY.parquet

    !aws s3 cp --recursive --quiet /mnt/workspace/Data/NY.parquet s3://dse-weather/NY.parquet

    !aws s3 ls s3://dse-weather/

    local_path=data_dir+'/'+parquet
    hdfs_path='/weather/'+parquet
    local_path,hdfs_path

    !hdfs dfs -copyFromLocal $local_path $hdfs_path

    !hdfs dfs -du /weather/
    parquet_path=hdfs_path

In [16]:
parquet_path = data_dir+'/'+parquet
!du -sh $parquet_path

 28M	../../Data/Weather/NY.parquet


In [17]:
%%time
df=sqlContext.read.parquet(parquet_path)
print(df.count())
df.show(5)

84199
+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
|USC00303452|       PRCP|1907|[00 00 00 00 60 5...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 5 rows

CPU times: user 3.43 ms, sys: 3.2 ms, total: 6.63 ms
Wall time: 4.32 s


In [18]:
from time import time
t=time()

N=sc.defaultParallelism
print('Number of executors=',N)
print('took',time()-t,'seconds')

Number of executors= 10
took 0.0011188983917236328 seconds


In [ ]:
%%time
STAT=computeStatistics(sqlContext,df)

In [40]:
STAT['TMAX'].keys()

dict_keys(['UnDef', 'mean', 'std', 'SortedVals', 'low100', 'high100', 'low1000', 'high1000', 'eigval', 'eigvec', 'E', 'NE', 'O', 'NO', 'Cov', 'Mean', 'Var'])

In [27]:
filename=data_dir+'/STAT_%s.pickle'%state
dump((STAT,STAT_Descriptions),open(filename,'wb'))
!ls -l $data_dir

total 97400
drwxr-xr-x  17 yoavfreund  staff       544 Apr  1 14:06 NY.parquet
-rw-r--r--   1 yoavfreund  staff  23182008 Apr  1 14:06 NY.tgz
drwxr-xr-x  20 yoavfreund  staff       640 Apr  1 13:57 Old
-rw-r--r--   1 yoavfreund  staff  25684524 Apr  1 14:31 STAT_NY.pickle


In [28]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TMAX 0.0
SNOW 852107.7058667188
SNWD 4464167.852118857
TMIN 319734.53150046256
PRCP 1184305.1228400553
TOBS 277719.0089381143


In [29]:
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  25684524 Apr  1 14:31 ../../Data/Weather/STAT_NY.pickle


In [30]:
!gzip -f ../../Data/Weather/STAT*.pickle
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  14948011 Apr  1 14:31 ../../Data/Weather/STAT_NY.pickle.gz


In [31]:
for state in ['NY']:
    command="aws s3  cp ../../Data/Weather/STAT_%s.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_%s.pickle.gz"%(state,state)
    print(command)
    !$command

aws s3  cp ../../Data/Weather/STAT_NY.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz
upload: ../../Data/Weather/STAT_NY.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz


In [32]:
!aws s3  ls s3://mas-dse-open/Weather/by_state/ | grep STAT

2018-04-01 14:32:56   14948011 STAT_NY.pickle.gz
2018-03-18 20:33:54   11717259 STAT_RI.pickle.gz
